In [1]:
"""Simple Vehicles Routing Problem (VRP).

   This is a sample using the routing library python wrapper to solve a VRP
   problem.
   A description of the problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.
"""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))

### Generate own data

In [2]:
import random

def generate_data_model(grid_size, num_locations, num_vehicles):
    """Generates the data for the problem with a grid and randomly placed locations."""
    data = {}
    
    # Generate the grid
    grid = [(x, y) for x in range(grid_size) for y in range(grid_size)]
    
    # Randomly select locations from the grid
    locations = random.sample(grid, num_locations)
    
    # Calculate distances between locations
    distance_matrix = []
    for i in range(num_locations):
        row = []
        for j in range(num_locations):
            if i == j:
                row.append(0)
            else:
                dist = abs(locations[i][0] - locations[j][0]) + abs(locations[i][1] - locations[j][1])
                row.append(dist)
        distance_matrix.append(row)
    
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = num_vehicles
    data['depot'] = 0  # Assuming the depot is located at index 0
    
    return data

### Test code

In [3]:
def initialize(data):
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                            data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        5000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    
    return routing,manager

### Normal run

In [4]:
# Instantiate the data problem.
grid_size=100
random_locations=100
num_vehicles=3

data = generate_data_model(grid_size,random_locations,num_vehicles)
# data = create_data_model()

In [5]:
routing,manager = initialize(data)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)
else:
    print('No solution found!')

Objective: 42428
Route for vehicle 0:
 0 ->  61 ->  98 ->  93 ->  92 ->  94 ->  77 ->  69 ->  76 ->  44 ->  50 ->  64 ->  39 ->  29 ->  91 ->  33 ->  83 ->  74 ->  65 ->  23 ->  21 ->  46 ->  7 ->  68 ->  36 ->  8 ->  2 ->  62 ->  42 ->  37 ->  32 ->  27 ->  80 ->  34 ->  53 ->  59 ->  19 -> 0
Distance of the route: 412m

Route for vehicle 1:
 0 ->  17 ->  81 ->  30 ->  82 ->  47 ->  10 ->  89 ->  67 ->  73 ->  15 ->  95 ->  4 ->  6 ->  58 ->  38 ->  72 ->  12 ->  45 ->  3 ->  75 ->  84 ->  97 ->  86 ->  66 ->  85 ->  11 ->  16 ->  22 ->  51 -> 0
Distance of the route: 404m

Route for vehicle 2:
 0 ->  63 ->  90 ->  54 ->  99 ->  79 ->  35 ->  14 ->  55 ->  56 ->  49 ->  41 ->  9 ->  13 ->  25 ->  24 ->  71 ->  5 ->  96 ->  70 ->  40 ->  78 ->  87 ->  20 ->  1 ->  57 ->  52 ->  18 ->  88 ->  60 ->  26 ->  31 ->  48 ->  28 ->  43 -> 0
Distance of the route: 412m

Maximum of the route distances: 412m


### Get the partial solution

In [6]:
routing,manager = initialize(data)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

search_parameters.time_limit.seconds = 1  # Example time limit in seconds
search_parameters.lns_time_limit.seconds = 1  # Example local search time limit

# Solve the problem.
partial_solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if partial_solution:
    print_solution(data, manager, routing, partial_solution)
else:
    print('No solution found!')

Objective: 81918
Route for vehicle 0:
 0 ->  16 ->  85 ->  11 ->  86 ->  66 ->  97 ->  92 ->  94 ->  69 ->  52 ->  76 ->  45 ->  50 ->  64 ->  12 ->  38 ->  6 ->  67 ->  59 ->  53 ->  34 ->  80 ->  47 ->  82 ->  30 ->  81 ->  19 ->  17 ->  15 ->  73 ->  10 ->  8 ->  32 ->  27 ->  42 ->  33 ->  46 ->  23 ->  39 ->  87 ->  78 ->  40 ->  70 ->  20 ->  13 ->  9 ->  49 ->  14 ->  18 ->  88 ->  31 ->  26 ->  48 ->  93 ->  98 ->  61 ->  43 -> 0
Distance of the route: 804m

Route for vehicle 1:
 0 ->  22 ->  51 -> 0
Distance of the route: 42m

Route for vehicle 2:
 0 ->  63 ->  90 ->  54 ->  99 ->  75 ->  55 ->  56 ->  41 ->  24 ->  71 ->  5 ->  96 ->  57 ->  1 ->  29 ->  91 ->  83 ->  74 ->  65 ->  21 ->  7 ->  68 ->  36 ->  62 ->  37 ->  2 ->  89 ->  4 ->  58 ->  44 ->  3 ->  60 ->  84 ->  79 ->  35 ->  25 ->  72 ->  77 ->  95 ->  28 -> 0
Distance of the route: 672m

Maximum of the route distances: 804m


### Method that returns the correct initial routes from a partial solution (estimation)

In [7]:
def convert_to_initial_solution(solution, manager, routing):
    initial_routes = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            if node_index != 0:
                route.append(node_index)
            index = solution.Value(routing.NextVar(index))
        initial_routes.append(route)
    return initial_routes

### Guided solver

In [ ]:
routing,manager = initialize(data)

partial_solution_convert = convert_to_initial_solution(partial_solution, manager, routing)

# Close model with the custom search parameters.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

routing.CloseModelWithParameters(search_parameters)

# Get initial solution from routes after closing the model.
initial_solution = routing.ReadAssignmentFromRoutes(partial_solution_convert, True)

# Solve the problem.
solution = routing.SolveFromAssignmentWithParameters(
    initial_solution, search_parameters)

# Print solution on console.
if partial_solution:
    print_solution(data, manager, routing, partial_solution)
else:
    print('No solution found!')

: 

: 